<h1><center>Data Extraction from PhonePe Pulse Repo<center></h1>

<b>Importing Libraries<b>


In [4]:
#Import necessary libraries
import pandas as pd
import mysql.connector as sql
import os
import git
from pathlib import  Path


<b>Clone Git Repository<b>

In [5]:
# #Specify the directories
# git_url = 'https://github.com/PhonePe/pulse.git'
# cloned_directory = 'data/pulse_data'

# #Clone
# git.Repo.clone_from(git_url, cloned_directory)

<h1><center>Data Processing</center></h1>

Goto the main entry point of the program

<b>Declaring all the directories and the data dictionaries</b>

Here we are declaring all the directories from which we have to extract the data
and the data dictionaries where we will save the data to change to dataframe

In [ ]:
#Directories
agg_insurance_dir = 'data/pulse_data/data/aggregated/insurance/country/india/state'
agg_transaction_dir = 'data/pulse_data/data/aggregated/transaction/country/india/state'
agg_user_dir = 'data/pulse_data/data/aggregated/user/country/india/state'

In [ ]:
#Data Dictionaries
agg_insurance_data = {'States':[],'Years':[],'Quarters':[],'Total_Policies':[],'Total_Amount':[]}


<b>Defining necessary functions</b>

1. read_json - This function takes a json file as parameter and returns the json data

2. save_to_csv - This function takes a dictionary and name of the file as parameter and converts the dictionary to a pandas dataframe and dataframe to a csv file

In [7]:
#Read JSON
def read_json(path):
    return pd.read_json(path)

In [8]:
#Save the dataframe to csv file
def save_to_csv(file_dict,filename):
    #Convert the dictionary to dataframe
    df = pd.DataFrame(file_dict)
    df.to_csv(f'data/csv_data/{filename}',index = False)

<b>Extraction Functions</b>

This block contains all the functions that extracts data from the JSON file and saves it to a dictionary.
Parameters are state, year, quarter and the json data


In [16]:
#Extract aggregated insurance
def agg_insurance(state,year,quarter,year_json):
    data = year_json['data']['transactionData']
    if(data):
        for i in data:
            count = i['paymentInstruments'][0]['count']
            amount = i['paymentInstruments'][0]['amount']
            
            if '-' in state:
                state = state.replace('-',' ')
            agg_insurance_data['States'] .append(state.capitalize())
            
            agg_insurance_data['Years'].append(year)
            agg_insurance_data['Quarters'].append(quarter)
            agg_insurance_data['Total_Policies'].append(count)
            agg_insurance_data['Total_Amount'].append(amount)
    


<b>Accumulated extraction functions</b>

This function makes a dictionary of the extraction functions so that it becomes easier to call each extraction function

In [10]:
#Function to accumulate all the functions and call respective function

#Check Invalid
def invalid_extraction_op():
    raise Exception("Invalid operation") 

#Call the respective functions
def perform_extraction(state,year,quarter,operation,year_json):
    etl_functions = {
    "agg_insurance": agg_insurance,
    }

    chosen_extraction_function = etl_functions.get(operation, invalid_extraction_op)

    return chosen_extraction_function(state,year,quarter,year_json)

<b>Function to iterate through the directories</b>

This function takes a directory path and the extraction function name as parameter.
It goes through the directories and sends the JSON file for extraction

In [17]:
#Function to iterate and get the JSON file
def iterate_through_files(directory,operation):
        states_list = os.listdir(directory)

        #Iterate through states list
        for state in states_list:
                #Get the state path
                state_path = f'{directory}//{state}'
                #List all the years
                state_year = os.listdir(state_path)
                #Iterate through year list
                for year in state_year:
                        #Get a year path
                        year_path = f'{state_path}//{year}'
                        #List all the files in the year folder
                        filename_list = []
                        for (dirpath, dirnames, files) in os.walk(year_path):
                                filename_list.extend(files)
                                break
                        #Iterate for each json file
                        for quarter_file in filename_list:
                                #Get the file path
                                quarter_path = f'{year_path}//{quarter_file}'
                                #Extract the quarter from the file name
                                quarter = int(Path(quarter_path).stem)
                                #Read the json file
                                year_json = read_json(quarter_path)
                                #Function call for each file to enter it in the data dictionary
                                perform_extraction(state,year,quarter,operation,year_json)


<h3><center>*********<u>Entry Point of the program</u>*********</center></h3>

<center>Run all the cell blocks above and the run this to start the process</center>

<b>Calling the functions to iterate through the directories</b>

Here we call the <i><u>iterate_through_files()</u></i> for all the directories.
Then the extracted data is saved into the data dictionaries.
After that the dictionary is saved to the csv file
This is the main entry point

In [18]:
#Function calls for iterating through the files

#Aggregated insurance
iterate_through_files(agg_insurance_dir,'agg_insurance')
save_to_csv(agg_insurance_data,'Aggregated_insurance_table.csv')